# TP2 - Organización de Datos
#### Notebook principal

<hr>

### Notebooks utilizados:

- ***pre_processing:*** notebook para el manejo inicial de los dataframes.
- ***feature_generation:*** primer etapa del pipeline. En este notebook se generarán nuevos features para luego, realizar un proceso de selección de los mejores features para cada modelo.
- ***feature_selection*** segunda etapa, donde se buscara encontrar los features con mayor importancia, es decir aquellos que aporten mayor informacion.
- ***parameter_tuning:*** tercer etapa, notebook donde se tunean los parámetros para cada modelo.
- ***predict:*** finalmente, una vez obtenidos los mejores parametros y features para cada modelo, este notebook se encargará de generar el csv con las predicciones finales para el modelo que se le indique.

<hr>


In [1]:
import pandas as pd
import numpy as np
import math

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

seed = 42

In [2]:
import nbimporter

import pre_processing
import feature_generation
import feature_selection
import parameter_tuning
import predict

Importing Jupyter notebook from pre_processing.ipynb
Importing Jupyter notebook from feature_generation.ipynb
Importing Jupyter notebook from feature_selection.ipynb
Importing Jupyter notebook from parameter_tuning.ipynb
Importing Jupyter notebook from predict.ipynb


In [3]:
def escribir_respuesta(ids,predicciones):
    with open("respuesta.csv",'w') as archivo:
        archivo.write("id,target\n")
        for i in range(len(ids)):
            linea = f"{int(ids[i])},{predicciones[i]}"
            archivo.write(f"{linea}\n")

<hr>

# Resultados obtenidos

# COMPARANDO VS LO ANTERIOR
### XGBOOST

In [4]:
import xgboost

In [10]:
train,test = pre_processing.load_featured_datasets()

In [11]:
train['precio'] = train['precio'].map(lambda x: math.log(x))

In [12]:
train_old = train.copy()
train_new = train

In [13]:
train_new = feature_selection.get_selected_dataframe(train_new)
train_old = feature_selection.get_old_ones(train_old)

In [16]:
# Entrenamos para lo anterior...

In [17]:
X = train_old.drop('precio', axis=1).values
Y = train_old['precio'].values

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=seed)

In [19]:
reg = xgboost.XGBRegressor(max_depth=12,n_estimators=140 ,learning_rate=0.08, verbosity=2,subsample=0.9, min_child_weight=20, n_jobs=2)
reg.fit(X_train,Y_train)

[22:26:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:26:13] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 0 pruned nodes, max_depth=7
[22:26:13] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 0 pruned nodes, max_depth=8
[22:26:14] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 74 extra nodes, 0 pruned nodes, max_depth=8
[22:26:15] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 70 extra nodes, 0 pruned nodes, max_depth=8
[22:26:16] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 78 extra nodes, 0 pruned nodes, max_depth=8
[22:26:17] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 74 extra nodes, 0 pruned nodes, max_depth=7
[22:26:17] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 82 extra nodes

[22:27:35] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1146 extra nodes, 0 pruned nodes, max_depth=12
[22:27:37] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1790 extra nodes, 0 pruned nodes, max_depth=12
[22:27:39] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1152 extra nodes, 0 pruned nodes, max_depth=12
[22:27:40] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 914 extra nodes, 0 pruned nodes, max_depth=12
[22:27:41] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1212 extra nodes, 0 pruned nodes, max_depth=12
[22:27:43] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1224 extra nodes, 0 pruned nodes, max_depth=12
[22:27:44] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1550 extra nodes, 0 pruned nodes, max_depth=12
[22:27:45] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1

[22:29:07] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1476 extra nodes, 0 pruned nodes, max_depth=12
[22:29:08] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 876 extra nodes, 0 pruned nodes, max_depth=12
[22:29:10] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 342 extra nodes, 0 pruned nodes, max_depth=12
[22:29:11] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 850 extra nodes, 0 pruned nodes, max_depth=12
[22:29:12] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 828 extra nodes, 0 pruned nodes, max_depth=12
[22:29:13] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1280 extra nodes, 0 pruned nodes, max_depth=12
[22:29:14] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 634 extra nodes, 0 pruned nodes, max_depth=12
[22:29:15] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roo

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.08, max_delta_step=0,
             max_depth=12, min_child_weight=20, missing=None, n_estimators=140,
             n_jobs=2, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=0.9, verbosity=2)

In [21]:
Y_pred = reg.predict(X_val)

f = np.vectorize(math.exp)
Y_pred = f(Y_pred)
Y_val = f(Y_val)
mean_absolute_error(Y_val,Y_pred)

483851.3853763512

In [22]:
# Entrenamos para las features seleccionadas.

In [23]:
X = train_new.drop('precio', axis=1).values
Y = train_new['precio'].values

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=seed)

In [24]:
reg = xgboost.XGBRegressor(max_depth=12,n_estimators=140 ,learning_rate=0.08, verbosity=2,subsample=0.9, min_child_weight=20, n_jobs=2)
reg.fit(X_train,Y_train)

[22:30:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:30:45] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=8
[22:30:47] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=8
[22:30:48] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 74 extra nodes, 0 pruned nodes, max_depth=8
[22:30:50] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 74 extra nodes, 0 pruned nodes, max_depth=8
[22:30:51] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 64 extra nodes, 0 pruned nodes, max_depth=7
[22:30:52] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 78 extra nodes, 0 pruned nodes, max_depth=8
[22:30:53] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 80 extra nodes

[22:32:33] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1148 extra nodes, 0 pruned nodes, max_depth=12
[22:32:35] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1218 extra nodes, 0 pruned nodes, max_depth=12
[22:32:36] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1410 extra nodes, 0 pruned nodes, max_depth=12
[22:32:38] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1138 extra nodes, 0 pruned nodes, max_depth=12
[22:32:39] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1114 extra nodes, 0 pruned nodes, max_depth=12
[22:32:41] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1262 extra nodes, 0 pruned nodes, max_depth=12
[22:32:42] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1554 extra nodes, 0 pruned nodes, max_depth=12
[22:32:44] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 

[22:34:07] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1136 extra nodes, 0 pruned nodes, max_depth=12
[22:34:08] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1068 extra nodes, 0 pruned nodes, max_depth=12
[22:34:10] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 594 extra nodes, 0 pruned nodes, max_depth=12
[22:34:11] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 616 extra nodes, 0 pruned nodes, max_depth=12
[22:34:13] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1002 extra nodes, 0 pruned nodes, max_depth=12
[22:34:14] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 886 extra nodes, 0 pruned nodes, max_depth=12
[22:34:16] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 484 extra nodes, 0 pruned nodes, max_depth=12
[22:34:17] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 ro

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.08, max_delta_step=0,
             max_depth=12, min_child_weight=20, missing=None, n_estimators=140,
             n_jobs=2, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=0.9, verbosity=2)

In [25]:
Y_pred = reg.predict(X_val)

f = np.vectorize(math.exp)
Y_pred = f(Y_pred)
Y_val = f(Y_val)
mean_absolute_error(Y_val,Y_pred)

479871.2188498117

### LightGBM

In [26]:
import lightgbm as lgb

In [32]:
# parametros que nos dieron el 481k
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mae',
    'max_depth': 14, 
    'learning_rate': 0.05,
    'verbose': 0, 
    'early_stopping_round': 200}
n_estimators=20000

In [27]:
train,test = pre_processing.load_featured_datasets()

In [28]:
train['precio'] = train['precio'].map(lambda x: math.log(x))

In [29]:
train_old = train.copy()
train_new = train

In [30]:
train_new = feature_selection.get_selected_dataframe(train_new)
train_old = feature_selection.get_old_ones(train_old)

In [31]:
# Entrenamos para lo anterior...

In [42]:
X = train_old.drop('precio', axis=1)
Y = train_old['precio']

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=seed)

In [43]:
d_train = lgb.Dataset(X_train.values, label=Y_train.values)
d_valid = lgb.Dataset(X_val.values, label=Y_val.values)
watchlist = [d_valid]
#reg = lgb.train(params, d_train, n_estimators, watchlist, verbose_eval=1)

In [37]:
Y_pred = reg.predict(X_val.values)

f = np.vectorize(math.exp)
Y_pred = f(Y_pred)
Y_val = f(Y_val.values)
mean_absolute_error(Y_val,Y_pred)

464286.9853739084

In [38]:
# Entrenamos para lo nuevo...

In [44]:
X = train_new.drop('precio', axis=1)
Y = train_new['precio']

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=seed)

In [45]:
d_train = lgb.Dataset(X_train.values, label=Y_train.values)
d_valid = lgb.Dataset(X_val.values, label=Y_val.values)
watchlist = [d_valid]
reg = lgb.train(params, d_train, n_estimators, watchlist, verbose_eval=1)

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's l1: 0.66422
Training until validation scores don't improve for 200 rounds
[2]	valid_0's l1: 0.638907
[3]	valid_0's l1: 0.615131
[4]	valid_0's l1: 0.592881
[5]	valid_0's l1: 0.571855
[6]	valid_0's l1: 0.55206
[7]	valid_0's l1: 0.53311
[8]	valid_0's l1: 0.515499
[9]	valid_0's l1: 0.499003
[10]	valid_0's l1: 0.483468
[11]	valid_0's l1: 0.46873
[12]	valid_0's l1: 0.454905
[13]	valid_0's l1: 0.441802
[14]	valid_0's l1: 0.429646
[15]	valid_0's l1: 0.417924
[16]	valid_0's l1: 0.407154
[17]	valid_0's l1: 0.396952
[18]	valid_0's l1: 0.3874
[19]	valid_0's l1: 0.378462
[20]	valid_0's l1: 0.370156
[21]	valid_0's l1: 0.362104
[22]	valid_0's l1: 0.354495
[23]	valid_0's l1: 0.347359
[24]	valid_0's l1: 0.340566
[25]	valid_0's l1: 0.334186
[26]	valid_0's l1: 0.328183
[27]	valid_0's l1: 0.322629
[28]	valid_0's l1: 0.317329
[29]	valid_0's l1: 0.312322
[30]	valid_0's l1: 0.307679
[31]	valid_0's l1: 0.303173
[32]	valid_0's l1: 0.299011
[33]	valid_0's l1: 0.294999
[34]	valid_0's l1: 0.291254
[

[291]	valid_0's l1: 0.208077
[292]	valid_0's l1: 0.208026
[293]	valid_0's l1: 0.207977
[294]	valid_0's l1: 0.207939
[295]	valid_0's l1: 0.207922
[296]	valid_0's l1: 0.207866
[297]	valid_0's l1: 0.207856
[298]	valid_0's l1: 0.207829
[299]	valid_0's l1: 0.207797
[300]	valid_0's l1: 0.207785
[301]	valid_0's l1: 0.207739
[302]	valid_0's l1: 0.207707
[303]	valid_0's l1: 0.207668
[304]	valid_0's l1: 0.207637
[305]	valid_0's l1: 0.207605
[306]	valid_0's l1: 0.207563
[307]	valid_0's l1: 0.207542
[308]	valid_0's l1: 0.207509
[309]	valid_0's l1: 0.207465
[310]	valid_0's l1: 0.207435
[311]	valid_0's l1: 0.207408
[312]	valid_0's l1: 0.207388
[313]	valid_0's l1: 0.207375
[314]	valid_0's l1: 0.20736
[315]	valid_0's l1: 0.207344
[316]	valid_0's l1: 0.207293
[317]	valid_0's l1: 0.207281
[318]	valid_0's l1: 0.207259
[319]	valid_0's l1: 0.207234
[320]	valid_0's l1: 0.207212
[321]	valid_0's l1: 0.207155
[322]	valid_0's l1: 0.207112
[323]	valid_0's l1: 0.20708
[324]	valid_0's l1: 0.207059
[325]	valid_0's 

[582]	valid_0's l1: 0.202118
[583]	valid_0's l1: 0.202109
[584]	valid_0's l1: 0.202085
[585]	valid_0's l1: 0.202062
[586]	valid_0's l1: 0.20204
[587]	valid_0's l1: 0.202025
[588]	valid_0's l1: 0.202015
[589]	valid_0's l1: 0.201995
[590]	valid_0's l1: 0.201984
[591]	valid_0's l1: 0.201972
[592]	valid_0's l1: 0.201963
[593]	valid_0's l1: 0.201942
[594]	valid_0's l1: 0.201935
[595]	valid_0's l1: 0.201938
[596]	valid_0's l1: 0.201916
[597]	valid_0's l1: 0.201904
[598]	valid_0's l1: 0.201895
[599]	valid_0's l1: 0.201887
[600]	valid_0's l1: 0.201884
[601]	valid_0's l1: 0.201876
[602]	valid_0's l1: 0.201845
[603]	valid_0's l1: 0.201829
[604]	valid_0's l1: 0.201817
[605]	valid_0's l1: 0.201799
[606]	valid_0's l1: 0.20178
[607]	valid_0's l1: 0.201751
[608]	valid_0's l1: 0.20173
[609]	valid_0's l1: 0.20171
[610]	valid_0's l1: 0.201696
[611]	valid_0's l1: 0.201691
[612]	valid_0's l1: 0.201666
[613]	valid_0's l1: 0.20166
[614]	valid_0's l1: 0.201644
[615]	valid_0's l1: 0.201633
[616]	valid_0's l1:

[866]	valid_0's l1: 0.198954
[867]	valid_0's l1: 0.198946
[868]	valid_0's l1: 0.198939
[869]	valid_0's l1: 0.198926
[870]	valid_0's l1: 0.198922
[871]	valid_0's l1: 0.198908
[872]	valid_0's l1: 0.198897
[873]	valid_0's l1: 0.198899
[874]	valid_0's l1: 0.1989
[875]	valid_0's l1: 0.198894
[876]	valid_0's l1: 0.198885
[877]	valid_0's l1: 0.19888
[878]	valid_0's l1: 0.198876
[879]	valid_0's l1: 0.198874
[880]	valid_0's l1: 0.198863
[881]	valid_0's l1: 0.198857
[882]	valid_0's l1: 0.198847
[883]	valid_0's l1: 0.198835
[884]	valid_0's l1: 0.198824
[885]	valid_0's l1: 0.198821
[886]	valid_0's l1: 0.198816
[887]	valid_0's l1: 0.198805
[888]	valid_0's l1: 0.198795
[889]	valid_0's l1: 0.198792
[890]	valid_0's l1: 0.19878
[891]	valid_0's l1: 0.198747
[892]	valid_0's l1: 0.198744
[893]	valid_0's l1: 0.19874
[894]	valid_0's l1: 0.198732
[895]	valid_0's l1: 0.198729
[896]	valid_0's l1: 0.198718
[897]	valid_0's l1: 0.198711
[898]	valid_0's l1: 0.198697
[899]	valid_0's l1: 0.198672
[900]	valid_0's l1:

...

[18741]	valid_0's l1: 0.181686
[18742]	valid_0's l1: 0.181685
[18743]	valid_0's l1: 0.181685
[18744]	valid_0's l1: 0.181684
[18745]	valid_0's l1: 0.181684
[18746]	valid_0's l1: 0.181684
[18747]	valid_0's l1: 0.181683
[18748]	valid_0's l1: 0.181683
[18749]	valid_0's l1: 0.181683
[18750]	valid_0's l1: 0.181682
[18751]	valid_0's l1: 0.181683
[18752]	valid_0's l1: 0.181681
[18753]	valid_0's l1: 0.181682
[18754]	valid_0's l1: 0.18168
[18755]	valid_0's l1: 0.18168
[18756]	valid_0's l1: 0.181679
[18757]	valid_0's l1: 0.18168
[18758]	valid_0's l1: 0.181681
[18759]	valid_0's l1: 0.181682
[18760]	valid_0's l1: 0.181682
[18761]	valid_0's l1: 0.181683
[18762]	valid_0's l1: 0.181683
[18763]	valid_0's l1: 0.181683
[18764]	valid_0's l1: 0.181685
[18765]	valid_0's l1: 0.181684
[18766]	valid_0's l1: 0.181685
[18767]	valid_0's l1: 0.181684
[18768]	valid_0's l1: 0.181684
[18769]	valid_0's l1: 0.181684
[18770]	valid_0's l1: 0.181685
[18771]	valid_0's l1: 0.181684
[18772]	valid_0's l1: 0.181685
[18773]	val

[19013]	valid_0's l1: 0.18166
[19014]	valid_0's l1: 0.181658
[19015]	valid_0's l1: 0.181657
[19016]	valid_0's l1: 0.181657
[19017]	valid_0's l1: 0.181656
[19018]	valid_0's l1: 0.181657
[19019]	valid_0's l1: 0.181657
[19020]	valid_0's l1: 0.181655
[19021]	valid_0's l1: 0.181656
[19022]	valid_0's l1: 0.181657
[19023]	valid_0's l1: 0.181656
[19024]	valid_0's l1: 0.181655
[19025]	valid_0's l1: 0.181656
[19026]	valid_0's l1: 0.181656
[19027]	valid_0's l1: 0.181656
[19028]	valid_0's l1: 0.181656
[19029]	valid_0's l1: 0.181655
[19030]	valid_0's l1: 0.181654
[19031]	valid_0's l1: 0.181655
[19032]	valid_0's l1: 0.181655
[19033]	valid_0's l1: 0.181655
[19034]	valid_0's l1: 0.181655
[19035]	valid_0's l1: 0.181656
[19036]	valid_0's l1: 0.181656
[19037]	valid_0's l1: 0.181655
[19038]	valid_0's l1: 0.181654
[19039]	valid_0's l1: 0.181654
[19040]	valid_0's l1: 0.181654
[19041]	valid_0's l1: 0.181653
[19042]	valid_0's l1: 0.181652
[19043]	valid_0's l1: 0.181652
[19044]	valid_0's l1: 0.181654
[19045]	v

[19280]	valid_0's l1: 0.181636
[19281]	valid_0's l1: 0.181636
[19282]	valid_0's l1: 0.181636
[19283]	valid_0's l1: 0.181637
[19284]	valid_0's l1: 0.181638
[19285]	valid_0's l1: 0.181638
[19286]	valid_0's l1: 0.181639
[19287]	valid_0's l1: 0.18164
[19288]	valid_0's l1: 0.18164
[19289]	valid_0's l1: 0.18164
[19290]	valid_0's l1: 0.181639
[19291]	valid_0's l1: 0.181639
[19292]	valid_0's l1: 0.181639
[19293]	valid_0's l1: 0.181638
[19294]	valid_0's l1: 0.181638
[19295]	valid_0's l1: 0.181638
[19296]	valid_0's l1: 0.181639
[19297]	valid_0's l1: 0.181639
[19298]	valid_0's l1: 0.181639
[19299]	valid_0's l1: 0.181639
[19300]	valid_0's l1: 0.181639
[19301]	valid_0's l1: 0.18164
[19302]	valid_0's l1: 0.18164
[19303]	valid_0's l1: 0.18164
[19304]	valid_0's l1: 0.18164
[19305]	valid_0's l1: 0.18164
[19306]	valid_0's l1: 0.181641
[19307]	valid_0's l1: 0.181641
[19308]	valid_0's l1: 0.18164
[19309]	valid_0's l1: 0.18164
[19310]	valid_0's l1: 0.181641
[19311]	valid_0's l1: 0.18164
[19312]	valid_0's l

[19549]	valid_0's l1: 0.1816
[19550]	valid_0's l1: 0.181599
[19551]	valid_0's l1: 0.181599
[19552]	valid_0's l1: 0.1816
[19553]	valid_0's l1: 0.1816
[19554]	valid_0's l1: 0.1816
[19555]	valid_0's l1: 0.1816
[19556]	valid_0's l1: 0.181598
[19557]	valid_0's l1: 0.181599
[19558]	valid_0's l1: 0.181599
[19559]	valid_0's l1: 0.181598
[19560]	valid_0's l1: 0.181597
[19561]	valid_0's l1: 0.181597
[19562]	valid_0's l1: 0.181597
[19563]	valid_0's l1: 0.181597
[19564]	valid_0's l1: 0.181595
[19565]	valid_0's l1: 0.181595
[19566]	valid_0's l1: 0.181593
[19567]	valid_0's l1: 0.181592
[19568]	valid_0's l1: 0.181592
[19569]	valid_0's l1: 0.181591
[19570]	valid_0's l1: 0.181592
[19571]	valid_0's l1: 0.181591
[19572]	valid_0's l1: 0.18159
[19573]	valid_0's l1: 0.18159
[19574]	valid_0's l1: 0.18159
[19575]	valid_0's l1: 0.18159
[19576]	valid_0's l1: 0.181589
[19577]	valid_0's l1: 0.181589
[19578]	valid_0's l1: 0.181589
[19579]	valid_0's l1: 0.18159
[19580]	valid_0's l1: 0.181589
[19581]	valid_0's l1: 0

[19817]	valid_0's l1: 0.18159
[19818]	valid_0's l1: 0.181589
[19819]	valid_0's l1: 0.18159
[19820]	valid_0's l1: 0.18159
[19821]	valid_0's l1: 0.181591
[19822]	valid_0's l1: 0.181591
[19823]	valid_0's l1: 0.181591
[19824]	valid_0's l1: 0.181589
[19825]	valid_0's l1: 0.181589
[19826]	valid_0's l1: 0.181589
[19827]	valid_0's l1: 0.181588
[19828]	valid_0's l1: 0.181587
[19829]	valid_0's l1: 0.181587
[19830]	valid_0's l1: 0.181587
[19831]	valid_0's l1: 0.181587
[19832]	valid_0's l1: 0.181587
[19833]	valid_0's l1: 0.181586
[19834]	valid_0's l1: 0.181586
[19835]	valid_0's l1: 0.181588
[19836]	valid_0's l1: 0.181588
[19837]	valid_0's l1: 0.181589
[19838]	valid_0's l1: 0.181589
[19839]	valid_0's l1: 0.181589
[19840]	valid_0's l1: 0.181589
[19841]	valid_0's l1: 0.181588
[19842]	valid_0's l1: 0.181587
[19843]	valid_0's l1: 0.181588
[19844]	valid_0's l1: 0.181587
[19845]	valid_0's l1: 0.181587
[19846]	valid_0's l1: 0.181587
[19847]	valid_0's l1: 0.181587
[19848]	valid_0's l1: 0.181588
[19849]	val

In [ ]:
Y_pred = reg.predict(X_val.values)

f = np.vectorize(math.exp)
Y_pred = f(Y_pred)
Y_val = f(Y_val.values)
mean_absolute_error(Y_val,Y_pred)

# Mejor submit hasta la fecha -- 19/11

In [4]:
import lightgbm as lgb
from datetime import datetime

In [5]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mae',
    'max_depth': 14,
    'num_leaves': 120,
    'learning_rate': 0.02,
    'verbose': 0, 
    'early_stopping_round': 1000}
n_estimators=99999999

In [6]:
train,test = pre_processing.load_featured_datasets()

In [7]:
train['precio'] = train['precio'].map(lambda x: math.log(x))

In [8]:
train_selected = feature_selection.get_selected_dataframe(train)

In [9]:
X = train_selected.drop('precio', axis=1)
Y = train_selected['precio']

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.1, random_state=seed)

In [10]:
print(datetime.now())

2019-11-19 23:15:12.446117


In [11]:
d_train = lgb.Dataset(X_train.values, label=Y_train.values)
d_valid = lgb.Dataset(X_val.values, label=Y_val.values)
watchlist = [d_valid]
reg = lgb.train(params, d_train, n_estimators, watchlist, verbose_eval=500)

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 1000 rounds
[500]	valid_0's l1: 0.197949
[1000]	valid_0's l1: 0.192564
[1500]	valid_0's l1: 0.189929
[2000]	valid_0's l1: 0.188237
[2500]	valid_0's l1: 0.186912
[3000]	valid_0's l1: 0.185966
[3500]	valid_0's l1: 0.185192
[4000]	valid_0's l1: 0.184344
[4500]	valid_0's l1: 0.183718
[5000]	valid_0's l1: 0.183126
[5500]	valid_0's l1: 0.182645
[6000]	valid_0's l1: 0.182254
[6500]	valid_0's l1: 0.181833
[7000]	valid_0's l1: 0.181479
[7500]	valid_0's l1: 0.181153
[8000]	valid_0's l1: 0.180848
[8500]	valid_0's l1: 0.1806
[9000]	valid_0's l1: 0.180359
[9500]	valid_0's l1: 0.180107
[10000]	valid_0's l1: 0.179954
[10500]	valid_0's l1: 0.179728
[11000]	valid_0's l1: 0.179515
[11500]	valid_0's l1: 0.179334
[12000]	valid_0's l1: 0.179182
[12500]	valid_0's l1: 0.17907
[13000]	valid_0's l1: 0.178926
[13500]	valid_0's l1: 0.178839
[14000]	valid_0's l1: 0.178763
[14500]	valid_0's l1: 0.178636
[15000]	valid_0's l1: 0.178544
[15500]	valid_0's l1: 0.178446

In [12]:
print(datetime.now())

2019-11-19 23:32:58.995210


In [13]:
Y_pred = reg.predict(X_val.values)

f = np.vectorize(math.exp)
Y_pred = f(Y_pred)
Y_val = f(Y_val.values)
mean_absolute_error(Y_val,Y_pred)

449620.60086473916

In [14]:
# Preparamos respuesta para Kaggle

In [17]:
test_selected = feature_selection.get_selected_dataframe(test, precio=False)

In [18]:
ids = test_selected.index.values
X_test = test_selected.values
test_predict = reg.predict(X_test)
f = np.vectorize(math.exp)
test_predict = f(test_predict)
escribir_respuesta(ids, test_predict)

In [22]:
df = X_val.copy()
df['precio'] = Y_val
df['predicted'] = Y_pred

In [23]:
df['error'] = df.apply(lambda x: abs(x['precio'] - x['predicted']), axis=1)

In [24]:
df['error'].describe()

count    2.400000e+04
mean     4.496206e+05
std      6.882597e+05
min      5.513569e+00
25%      7.511666e+04
50%      2.096197e+05
75%      5.369516e+05
max      1.099456e+07
Name: error, dtype: float64

In [25]:
df.shape

(24000, 144)

In [47]:
mucho_error = df.loc[df['error'] > 6000000]
mucho_error.shape[0]

30

In [51]:
mucho_error['es_Distrito Federal'].sum()

10

In [9]:
import lightgbm as lgb

In [10]:
train,test = pre_processing.load_featured_datasets()

In [11]:
train['precio'] = train['precio'].map(lambda x: math.log(x))

In [12]:
train_selected = feature_selection.get_selected_dataframe(train)
test_selected = feature_selection.get_selected_dataframe(test, precio=False)

In [13]:
X = train_selected.drop('precio', axis=1)
Y = train_selected['precio']

In [14]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=seed)

In [17]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mae',
    'max_depth': 14,
    'num_leaves': 90,
    'learning_rate': 0.2,
    'verbose': 0, 
    #'bagging_fraction':0.9,
    #'feature_fraction':0.8,
    'early_stopping_round': 50,
    'n_jobs':2}
n_estimators=20000

In [20]:
d_train = lgb.Dataset(X_train.values, label=Y_train.values)
d_valid = lgb.Dataset(X_val.values, label=Y_val.values)
watchlist = [d_valid]
reg = lgb.train(params, d_train, n_estimators, watchlist, verbose_eval=1)

[1]	valid_0's l1: 0.66422
Training until validation scores don't improve for 200 rounds
[2]	valid_0's l1: 0.638907
[3]	valid_0's l1: 0.615131
[4]	valid_0's l1: 0.592881
[5]	valid_0's l1: 0.571855
[6]	valid_0's l1: 0.55206
[7]	valid_0's l1: 0.53311
[8]	valid_0's l1: 0.515499
[9]	valid_0's l1: 0.499003
[10]	valid_0's l1: 0.483468
[11]	valid_0's l1: 0.46873
[12]	valid_0's l1: 0.454905
[13]	valid_0's l1: 0.441802
[14]	valid_0's l1: 0.429646
[15]	valid_0's l1: 0.417924
[16]	valid_0's l1: 0.407154
[17]	valid_0's l1: 0.396952
[18]	valid_0's l1: 0.3874
[19]	valid_0's l1: 0.378462
[20]	valid_0's l1: 0.370156
[21]	valid_0's l1: 0.362104
[22]	valid_0's l1: 0.354495
[23]	valid_0's l1: 0.347359
[24]	valid_0's l1: 0.340566
[25]	valid_0's l1: 0.334186
[26]	valid_0's l1: 0.328183
[27]	valid_0's l1: 0.322629
[28]	valid_0's l1: 0.317329
[29]	valid_0's l1: 0.312322
[30]	valid_0's l1: 0.307679
[31]	valid_0's l1: 0.303173
[32]	valid_0's l1: 0.299011
[33]	valid_0's l1: 0.294999
[34]	valid_0's l1: 0.291254
[

[292]	valid_0's l1: 0.208026
[293]	valid_0's l1: 0.207977
[294]	valid_0's l1: 0.207939
[295]	valid_0's l1: 0.207922
[296]	valid_0's l1: 0.207866
[297]	valid_0's l1: 0.207856
[298]	valid_0's l1: 0.207829
[299]	valid_0's l1: 0.207797
[300]	valid_0's l1: 0.207785
[301]	valid_0's l1: 0.207739
[302]	valid_0's l1: 0.207707
[303]	valid_0's l1: 0.207668
[304]	valid_0's l1: 0.207637
[305]	valid_0's l1: 0.207605
[306]	valid_0's l1: 0.207563
[307]	valid_0's l1: 0.207542
[308]	valid_0's l1: 0.207509
[309]	valid_0's l1: 0.207465
[310]	valid_0's l1: 0.207435
[311]	valid_0's l1: 0.207408
[312]	valid_0's l1: 0.207388
[313]	valid_0's l1: 0.207375
[314]	valid_0's l1: 0.20736
[315]	valid_0's l1: 0.207344
[316]	valid_0's l1: 0.207293
[317]	valid_0's l1: 0.207281
[318]	valid_0's l1: 0.207259
[319]	valid_0's l1: 0.207234
[320]	valid_0's l1: 0.207212
[321]	valid_0's l1: 0.207155
[322]	valid_0's l1: 0.207112
[323]	valid_0's l1: 0.20708
[324]	valid_0's l1: 0.207059
[325]	valid_0's l1: 0.207013
[326]	valid_0's 

[583]	valid_0's l1: 0.202109
[584]	valid_0's l1: 0.202085
[585]	valid_0's l1: 0.202062
[586]	valid_0's l1: 0.20204
[587]	valid_0's l1: 0.202025
[588]	valid_0's l1: 0.202015
[589]	valid_0's l1: 0.201995
[590]	valid_0's l1: 0.201984
[591]	valid_0's l1: 0.201972
[592]	valid_0's l1: 0.201963
[593]	valid_0's l1: 0.201942
[594]	valid_0's l1: 0.201935
[595]	valid_0's l1: 0.201938
[596]	valid_0's l1: 0.201916
[597]	valid_0's l1: 0.201904
[598]	valid_0's l1: 0.201895
[599]	valid_0's l1: 0.201887
[600]	valid_0's l1: 0.201884
[601]	valid_0's l1: 0.201876
[602]	valid_0's l1: 0.201845
[603]	valid_0's l1: 0.201829
[604]	valid_0's l1: 0.201817
[605]	valid_0's l1: 0.201799
[606]	valid_0's l1: 0.20178
[607]	valid_0's l1: 0.201751
[608]	valid_0's l1: 0.20173
[609]	valid_0's l1: 0.20171
[610]	valid_0's l1: 0.201696
[611]	valid_0's l1: 0.201691
[612]	valid_0's l1: 0.201666
[613]	valid_0's l1: 0.20166
[614]	valid_0's l1: 0.201644
[615]	valid_0's l1: 0.201633
[616]	valid_0's l1: 0.201613
[617]	valid_0's l1:

[871]	valid_0's l1: 0.198908
[872]	valid_0's l1: 0.198897
[873]	valid_0's l1: 0.198899
[874]	valid_0's l1: 0.1989
[875]	valid_0's l1: 0.198894
[876]	valid_0's l1: 0.198885
[877]	valid_0's l1: 0.19888
[878]	valid_0's l1: 0.198876
[879]	valid_0's l1: 0.198874
[880]	valid_0's l1: 0.198863
[881]	valid_0's l1: 0.198857
[882]	valid_0's l1: 0.198847
[883]	valid_0's l1: 0.198835
[884]	valid_0's l1: 0.198824
[885]	valid_0's l1: 0.198821
[886]	valid_0's l1: 0.198816
[887]	valid_0's l1: 0.198805
[888]	valid_0's l1: 0.198795
[889]	valid_0's l1: 0.198792
[890]	valid_0's l1: 0.19878
[891]	valid_0's l1: 0.198747
[892]	valid_0's l1: 0.198744
[893]	valid_0's l1: 0.19874
[894]	valid_0's l1: 0.198732
[895]	valid_0's l1: 0.198729
[896]	valid_0's l1: 0.198718
[897]	valid_0's l1: 0.198711
[898]	valid_0's l1: 0.198697
[899]	valid_0's l1: 0.198672
[900]	valid_0's l1: 0.198668
[901]	valid_0's l1: 0.198668
[902]	valid_0's l1: 0.198663
[903]	valid_0's l1: 0.198656
[904]	valid_0's l1: 0.19865
[905]	valid_0's l1: 

[1151]	valid_0's l1: 0.196882
[1152]	valid_0's l1: 0.196879
[1153]	valid_0's l1: 0.196874
[1154]	valid_0's l1: 0.196859
[1155]	valid_0's l1: 0.196856
[1156]	valid_0's l1: 0.19685
[1157]	valid_0's l1: 0.196839
[1158]	valid_0's l1: 0.196836
[1159]	valid_0's l1: 0.196834
[1160]	valid_0's l1: 0.196828
[1161]	valid_0's l1: 0.196821
[1162]	valid_0's l1: 0.196823
[1163]	valid_0's l1: 0.196817
[1164]	valid_0's l1: 0.196809
[1165]	valid_0's l1: 0.196804
[1166]	valid_0's l1: 0.196799
[1167]	valid_0's l1: 0.196794
[1168]	valid_0's l1: 0.196784
[1169]	valid_0's l1: 0.196777
[1170]	valid_0's l1: 0.196768
[1171]	valid_0's l1: 0.196765
[1172]	valid_0's l1: 0.196759
[1173]	valid_0's l1: 0.196758
[1174]	valid_0's l1: 0.196753
[1175]	valid_0's l1: 0.196748
[1176]	valid_0's l1: 0.196736
[1177]	valid_0's l1: 0.196722
[1178]	valid_0's l1: 0.196713
[1179]	valid_0's l1: 0.196705
[1180]	valid_0's l1: 0.196698
[1181]	valid_0's l1: 0.196694
[1182]	valid_0's l1: 0.196678
[1183]	valid_0's l1: 0.196664
[1184]	vali

...

[18738]	valid_0's l1: 0.181687
[18739]	valid_0's l1: 0.181685
[18740]	valid_0's l1: 0.181685
[18741]	valid_0's l1: 0.181686
[18742]	valid_0's l1: 0.181685
[18743]	valid_0's l1: 0.181685
[18744]	valid_0's l1: 0.181684
[18745]	valid_0's l1: 0.181684
[18746]	valid_0's l1: 0.181684
[18747]	valid_0's l1: 0.181683
[18748]	valid_0's l1: 0.181683
[18749]	valid_0's l1: 0.181683
[18750]	valid_0's l1: 0.181682
[18751]	valid_0's l1: 0.181683
[18752]	valid_0's l1: 0.181681
[18753]	valid_0's l1: 0.181682
[18754]	valid_0's l1: 0.18168
[18755]	valid_0's l1: 0.18168
[18756]	valid_0's l1: 0.181679
[18757]	valid_0's l1: 0.18168
[18758]	valid_0's l1: 0.181681
[18759]	valid_0's l1: 0.181682
[18760]	valid_0's l1: 0.181682
[18761]	valid_0's l1: 0.181683
[18762]	valid_0's l1: 0.181683
[18763]	valid_0's l1: 0.181683
[18764]	valid_0's l1: 0.181685
[18765]	valid_0's l1: 0.181684
[18766]	valid_0's l1: 0.181685
[18767]	valid_0's l1: 0.181684
[18768]	valid_0's l1: 0.181684
[18769]	valid_0's l1: 0.181684
[18770]	val

[19012]	valid_0's l1: 0.181661
[19013]	valid_0's l1: 0.18166
[19014]	valid_0's l1: 0.181658
[19015]	valid_0's l1: 0.181657
[19016]	valid_0's l1: 0.181657
[19017]	valid_0's l1: 0.181656
[19018]	valid_0's l1: 0.181657
[19019]	valid_0's l1: 0.181657
[19020]	valid_0's l1: 0.181655
[19021]	valid_0's l1: 0.181656
[19022]	valid_0's l1: 0.181657
[19023]	valid_0's l1: 0.181656
[19024]	valid_0's l1: 0.181655
[19025]	valid_0's l1: 0.181656
[19026]	valid_0's l1: 0.181656
[19027]	valid_0's l1: 0.181656
[19028]	valid_0's l1: 0.181656
[19029]	valid_0's l1: 0.181655
[19030]	valid_0's l1: 0.181654
[19031]	valid_0's l1: 0.181655
[19032]	valid_0's l1: 0.181655
[19033]	valid_0's l1: 0.181655
[19034]	valid_0's l1: 0.181655
[19035]	valid_0's l1: 0.181656
[19036]	valid_0's l1: 0.181656
[19037]	valid_0's l1: 0.181655
[19038]	valid_0's l1: 0.181654
[19039]	valid_0's l1: 0.181654
[19040]	valid_0's l1: 0.181654
[19041]	valid_0's l1: 0.181653
[19042]	valid_0's l1: 0.181652
[19043]	valid_0's l1: 0.181652
[19044]	v

[19286]	valid_0's l1: 0.181639
[19287]	valid_0's l1: 0.18164
[19288]	valid_0's l1: 0.18164
[19289]	valid_0's l1: 0.18164
[19290]	valid_0's l1: 0.181639
[19291]	valid_0's l1: 0.181639
[19292]	valid_0's l1: 0.181639
[19293]	valid_0's l1: 0.181638
[19294]	valid_0's l1: 0.181638
[19295]	valid_0's l1: 0.181638
[19296]	valid_0's l1: 0.181639
[19297]	valid_0's l1: 0.181639
[19298]	valid_0's l1: 0.181639
[19299]	valid_0's l1: 0.181639
[19300]	valid_0's l1: 0.181639
[19301]	valid_0's l1: 0.18164
[19302]	valid_0's l1: 0.18164
[19303]	valid_0's l1: 0.18164
[19304]	valid_0's l1: 0.18164
[19305]	valid_0's l1: 0.18164
[19306]	valid_0's l1: 0.181641
[19307]	valid_0's l1: 0.181641
[19308]	valid_0's l1: 0.18164
[19309]	valid_0's l1: 0.18164
[19310]	valid_0's l1: 0.181641
[19311]	valid_0's l1: 0.18164
[19312]	valid_0's l1: 0.181641
[19313]	valid_0's l1: 0.181641
[19314]	valid_0's l1: 0.181642
[19315]	valid_0's l1: 0.181641
[19316]	valid_0's l1: 0.181641
[19317]	valid_0's l1: 0.18164
[19318]	valid_0's l1

[19561]	valid_0's l1: 0.181597
[19562]	valid_0's l1: 0.181597
[19563]	valid_0's l1: 0.181597
[19564]	valid_0's l1: 0.181595
[19565]	valid_0's l1: 0.181595
[19566]	valid_0's l1: 0.181593
[19567]	valid_0's l1: 0.181592
[19568]	valid_0's l1: 0.181592
[19569]	valid_0's l1: 0.181591
[19570]	valid_0's l1: 0.181592
[19571]	valid_0's l1: 0.181591
[19572]	valid_0's l1: 0.18159
[19573]	valid_0's l1: 0.18159
[19574]	valid_0's l1: 0.18159
[19575]	valid_0's l1: 0.18159
[19576]	valid_0's l1: 0.181589
[19577]	valid_0's l1: 0.181589
[19578]	valid_0's l1: 0.181589
[19579]	valid_0's l1: 0.18159
[19580]	valid_0's l1: 0.181589
[19581]	valid_0's l1: 0.18159
[19582]	valid_0's l1: 0.18159
[19583]	valid_0's l1: 0.18159
[19584]	valid_0's l1: 0.181591
[19585]	valid_0's l1: 0.18159
[19586]	valid_0's l1: 0.181592
[19587]	valid_0's l1: 0.181592
[19588]	valid_0's l1: 0.181592
[19589]	valid_0's l1: 0.181591
[19590]	valid_0's l1: 0.181592
[19591]	valid_0's l1: 0.181592
[19592]	valid_0's l1: 0.181592
[19593]	valid_0's

[19832]	valid_0's l1: 0.181587
[19833]	valid_0's l1: 0.181586
[19834]	valid_0's l1: 0.181586
[19835]	valid_0's l1: 0.181588
[19836]	valid_0's l1: 0.181588
[19837]	valid_0's l1: 0.181589
[19838]	valid_0's l1: 0.181589
[19839]	valid_0's l1: 0.181589
[19840]	valid_0's l1: 0.181589
[19841]	valid_0's l1: 0.181588
[19842]	valid_0's l1: 0.181587
[19843]	valid_0's l1: 0.181588
[19844]	valid_0's l1: 0.181587
[19845]	valid_0's l1: 0.181587
[19846]	valid_0's l1: 0.181587
[19847]	valid_0's l1: 0.181587
[19848]	valid_0's l1: 0.181588
[19849]	valid_0's l1: 0.181588
[19850]	valid_0's l1: 0.181588
[19851]	valid_0's l1: 0.181588
[19852]	valid_0's l1: 0.181587
[19853]	valid_0's l1: 0.181589
[19854]	valid_0's l1: 0.181589
[19855]	valid_0's l1: 0.181589
[19856]	valid_0's l1: 0.181587
[19857]	valid_0's l1: 0.181587
[19858]	valid_0's l1: 0.181587
[19859]	valid_0's l1: 0.181587
[19860]	valid_0's l1: 0.181587
[19861]	valid_0's l1: 0.181587
[19862]	valid_0's l1: 0.181587
[19863]	valid_0's l1: 0.181587
[19864]	

In [21]:
Y_pred = reg.predict(X_val.values)

f = np.vectorize(math.exp)
Y_pred = f(Y_pred)
Y_val = f(Y_val.values)
mean_absolute_error(Y_val,Y_pred)

456829.3894042744

In [14]:
# respuesta para kaggle

In [22]:
ids = test_selected.index.values
X_test = test_selected.values

In [23]:
test_predict = reg.predict(X_test)

f = np.vectorize(math.exp)
test_predict = f(test_predict)

In [24]:
escribir_respuesta(ids, test_predict)

In [19]:
# parametros que nos dieron el 481k
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mae',
    'max_depth': 14, 
    'learning_rate': 0.05,
    'verbose': 0, 
    'early_stopping_round': 200}
n_estimators=20000

### Modelo: Regresion lineal

In [ ]:
# ...

### Modelo: Regresion logistica

In [5]:
# ...

### Modelo: SVM

In [6]:
# ...

### Modelo: Decision Tree

In [7]:
# ...

### Modelo: RandomForest

In [4]:
from sklearn.ensemble import RandomForestRegressor

In [5]:
df = pre_processing.load_featured_appended_dataset()

In [10]:
features = feature_generation.get_features()

for feature in features:
    todas = []
    for opcion in features[feature]:
        valores = features[feature][opcion]
        for valor in valores:
            if (valor not in todas):
                todas.append(valor)
    features[feature]['todas'] = todas

In [11]:
# Como sabemos, este modelo no admite nulos, por lo que utilizaremos alguna tecnica de imputacion de los mismos
# para poder correrlo. Primero nos quedamos con las features mas importantes...
df = df[['antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales',
                        'idzona', 'lat', 'lng', 'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
                        'centroscomercialescercanos']\
                       +features["metros"][1]\
                       +features["tipodepropiedad"][0]\
                       +features["provincia"][6]\
                       +features["ciudad"]['todas']\
                       +features["fecha"][18]\
                       +features["descripcion"][0]\
                       +features["metricas"][2]\
                       +features["habitaciones"][0]\
                       +features["antiguedad"][1]\
                       +features["extras"][2]\
                       +features["volcanes"][0]\
                       +features["idzona"][5]\
                       +["precio"]]

In [ ]:
# Ahora utilizaremos una funcion definida en pre_processing para, utilizando otros regresores,
# predecir el valor que tienen las columnas con valores nulos.

df = pre_processing.fill_nans_lgb(df)

In [10]:
X = train.drop('precio', axis=1).values
Y = train['precio'].values
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2)

In [11]:
regressor = RandomForestRegressor(n_estimators = 100, random_state = seed, verbose=2, max_depth=10) 
regressor.fit(X_train, Y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.7s remaining:    0.0s


building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  5.1min finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=7, verbose=2,
                      warm_start=False)

In [12]:
from sklearn import metrics

In [15]:
y_pred = regressor.predict(X_val)
print('MAE: ', int(metrics.mean_absolute_error(Y_val, y_pred)))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


MAE:  688548


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.6s finished


In [16]:
y_pred2 = regressor.predict(X_train)
print('MAE: ', int(metrics.mean_absolute_error(Y_train, y_pred2)))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


MAE:  663067


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.2s finished


In [20]:
names = train.columns.to_list()
print(sorted(zip(map(lambda x: round(x, 4), regressor.feature_importances_), names), reverse=True))

[(0.4881, 'metroscubiertos'), (0.2571, 'ciudad_le'), (0.0352, 'ciudad_muycara'), (0.0317, 'banos'), (0.0158, 'tipodepropiedad_1_pol'), (0.0141, 'dia'), (0.0129, 'precio_promedio_metrocubierto_mes'), (0.0125, 'antiguedad'), (0.0113, 'garages'), (0.0105, 'servicio'), (0.0096, 'es_Veracruz'), (0.0093, 'metroscubiertos_mean'), (0.009, 'precio'), (0.0085, 'intercept_pol'), (0.0069, 'tipodepropiedad_2_pol'), (0.0065, 'tipodepropiedad_0_pol'), (0.005, 'habitaciones'), (0.0042, 'aniomes'), (0.0033, 'tipodepropiedad_3_pol'), (0.0031, 'ciudad_barata'), (0.0025, 'es_apart'), (0.0024, 'tipodepropiedad_4_pol'), (0.002, 'tipodepropiedad_le'), (0.002, 'ciudad_cara'), (0.0019, 'tipodepropiedad_8_ohe'), (0.0017, 'lujo'), (0.0017, 'aniomes_scaled'), (0.0015, 'mes'), (0.0015, 'es_casa'), (0.0014, 'tipodepropiedad_7_pol'), (0.0014, 'hab_binning_1_ohe'), (0.0013, 'provincia_10_ohe'), (0.0013, 'gimnasio'), (0.0012, 'parrilla'), (0.0011, 'piscina'), (0.0011, 'es_Distrito Federal'), (0.001, 'hab_binning_7_ohe

### Modelo: XGBoost

_Generacion del dataset de train con sus features_

In [4]:
import xgboost
from sklearn.model_selection import GridSearchCV

In [5]:
train,test = pre_processing.load_featured_datasets()

In [6]:
train['precio'] = train['precio'].map(lambda x: math.log(x))

In [7]:
train_selected = feature_selection.get_selected_dataframe(train)
test_selected = feature_selection.get_selected_dataframe(test, precio=False)

In [8]:
X = train_selected.drop('precio', axis=1).values
Y = train_selected['precio'].values

In [18]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2)

In [28]:
parametros = {
    'max_depth':[11,12,13,14,15],
    'n_estimators':[100,110,120,130,140],
    'learning_rate': [0.05,0.08,0.1,0.15,0.2,0.3],
    'subsample':[0.5,0.8,0.9,0.7],
    'min_child_weight':[5,10,15,20,30]
}

In [16]:
reg = xgboost.XGBRegressor(max_depth=12,n_estimators=140 ,learning_rate=0.08, verbosity=2,subsample=0.9, min_child_weight=20, n_jobs=2)
reg.fit(X_train,Y_train)

[20:19:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:19:24] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=8
[20:19:25] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=8
[20:19:26] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 68 extra nodes, 0 pruned nodes, max_depth=7
[20:19:27] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 68 extra nodes, 0 pruned nodes, max_depth=8
[20:19:28] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 82 extra nodes, 0 pruned nodes, max_depth=8
[20:19:29] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 80 extra nodes, 0 pruned nodes, max_depth=9
[20:19:31] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 88 extra nodes

[20:21:06] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1252 extra nodes, 0 pruned nodes, max_depth=12
[20:21:07] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1182 extra nodes, 0 pruned nodes, max_depth=12
[20:21:09] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 812 extra nodes, 0 pruned nodes, max_depth=12
[20:21:10] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1424 extra nodes, 0 pruned nodes, max_depth=12
[20:21:12] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1128 extra nodes, 0 pruned nodes, max_depth=12
[20:21:13] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1518 extra nodes, 0 pruned nodes, max_depth=12
[20:21:15] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1134 extra nodes, 0 pruned nodes, max_depth=12
[20:21:16] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1

[20:22:40] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1054 extra nodes, 0 pruned nodes, max_depth=12
[20:22:42] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 478 extra nodes, 0 pruned nodes, max_depth=12
[20:22:43] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 828 extra nodes, 0 pruned nodes, max_depth=12
[20:22:46] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 550 extra nodes, 0 pruned nodes, max_depth=12
[20:22:49] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 184 extra nodes, 0 pruned nodes, max_depth=12
[20:22:52] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 788 extra nodes, 0 pruned nodes, max_depth=12
[20:22:55] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1296 extra nodes, 0 pruned nodes, max_depth=12
[20:22:58] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roo

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.08, max_delta_step=0,
             max_depth=12, min_child_weight=20, missing=None, n_estimators=140,
             n_jobs=2, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=0.9, verbosity=2)

_Comprobacion contra el conjunto de validacion_

In [26]:
Y_pred = reg.predict(X_val)

f = np.vectorize(math.exp)
Y_pred = f(Y_pred)
Y_val = f(Y_val)
mean_absolute_error(Y_val,Y_pred)

473494.35926695546

In [12]:
# preparamos el csv de respuesta para kaggle

In [13]:
ids = test_selected.index.values
X_test = test_selected.values

In [14]:
test_predict = reg.predict(X_test)

f = np.vectorize(math.exp)
test_predict = f(test_predict)

In [15]:
escribir_respuesta(ids, test_predict)

### Modelo: CatBoost

In [ ]:
#...

### Modelo: LightGBM

In [4]:
import lightgbm as lgb

In [5]:
train,test = load_featured_datasets()

NameError: name 'load_featured_datasets' is not defined

In [75]:
features = feature_generation.get_features()

In [76]:
best_features = feature_selection.get_best_features_per_category()

In [98]:
features['metros']

{0: ['metroscubiertos_alt', 'metrostotales_alt'],
 1: ['metroscubiertos_alt',
  'metrostotales_alt',
  'metrostotales_confiables_alt'],
 2: ['metroscubiertos_i1', 'metrostotales_i1'],
 3: ['metroscubiertos_i1', 'metrostotales_i1', 'metrostotales_confiables_alt'],
 4: ['metroscubiertos_alt', 'metrostotales_i2'],
 5: ['metroscubiertos_alt',
  'metrostotales_i2',
  'metrostotales_confiables_alt']}

In [77]:
best_features

[('metros', 1),
 ('tipodepropiedad', 0),
 ('provincia', 6),
 ('ciudad', 2),
 ('fecha', 4),
 ('descripcion', 0),
 ('metricas', 2),
 ('habitaciones', 0),
 ('antiguedad', 1),
 ('extras', 2),
 ('volcanes', 0),
 ('idzona', 0)]

In [114]:
train_selected = train[['antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales',
                        'idzona', 'lat', 'lng', 'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
                        'centroscomercialescercanos']\
                       +features["metros"][1]\
                       +features["tipodepropiedad"][0]\
                       +features["provincia"][6]\
                       +features["ciudad"][2]\
                       +features["fecha"][4]\
                       +features["descripcion"][0]\
                       +features["metricas"][2]\
                       +features["habitaciones"][0]\
                       +features["antiguedad"][1]\
                       +features["extras"][2]\
                       +features["volcanes"][0]\
                       +features["idzona"][0]\
                       +["precio"]]

In [115]:
X = train_selected.drop('precio', axis=1).values
Y = train_selected['precio'].values

In [116]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=seed)

In [117]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mae',
    'max_depth': 14, 
    'learning_rate': 0.2,
    'verbose': 0, 
    'early_stopping_round': 50}
n_estimators=10000

In [118]:
d_train = lgb.Dataset(X_train, label=Y_train)
d_valid = lgb.Dataset(X_val, label=Y_val)
watchlist = [d_valid]
reg = lgb.train(params, d_train, n_estimators, watchlist, verbose_eval=1)

[1]	valid_0's l1: 1.37848e+06
Training until validation scores don't improve for 50 rounds
[2]	valid_0's l1: 1.20026e+06
[3]	valid_0's l1: 1.06646e+06
[4]	valid_0's l1: 964527
[5]	valid_0's l1: 886962
[6]	valid_0's l1: 825819
[7]	valid_0's l1: 780005
[8]	valid_0's l1: 742121
[9]	valid_0's l1: 713473
[10]	valid_0's l1: 687841
[11]	valid_0's l1: 669070
[12]	valid_0's l1: 654806
[13]	valid_0's l1: 642552
[14]	valid_0's l1: 632085
[15]	valid_0's l1: 623036
[16]	valid_0's l1: 616622
[17]	valid_0's l1: 611148
[18]	valid_0's l1: 606440
[19]	valid_0's l1: 600293
[20]	valid_0's l1: 596400
[21]	valid_0's l1: 592895
[22]	valid_0's l1: 589279
[23]	valid_0's l1: 586698
[24]	valid_0's l1: 583480
[25]	valid_0's l1: 580945
[26]	valid_0's l1: 579340
[27]	valid_0's l1: 576913
[28]	valid_0's l1: 575237
[29]	valid_0's l1: 572096
[30]	valid_0's l1: 570468
[31]	valid_0's l1: 568829
[32]	valid_0's l1: 567254
[33]	valid_0's l1: 566310
[34]	valid_0's l1: 565160
[35]	valid_0's l1: 563258
[36]	valid_0's l1: 5621

[313]	valid_0's l1: 513172
[314]	valid_0's l1: 513125
[315]	valid_0's l1: 513092
[316]	valid_0's l1: 513154
[317]	valid_0's l1: 513044
[318]	valid_0's l1: 513021
[319]	valid_0's l1: 512951
[320]	valid_0's l1: 512956
[321]	valid_0's l1: 512850
[322]	valid_0's l1: 512802
[323]	valid_0's l1: 512664
[324]	valid_0's l1: 512586
[325]	valid_0's l1: 512481
[326]	valid_0's l1: 512538
[327]	valid_0's l1: 512486
[328]	valid_0's l1: 512340
[329]	valid_0's l1: 512216
[330]	valid_0's l1: 512170
[331]	valid_0's l1: 512064
[332]	valid_0's l1: 511962
[333]	valid_0's l1: 511925
[334]	valid_0's l1: 511863
[335]	valid_0's l1: 511841
[336]	valid_0's l1: 511708
[337]	valid_0's l1: 511644
[338]	valid_0's l1: 511596
[339]	valid_0's l1: 511572
[340]	valid_0's l1: 511551
[341]	valid_0's l1: 511514
[342]	valid_0's l1: 511462
[343]	valid_0's l1: 511476
[344]	valid_0's l1: 511431
[345]	valid_0's l1: 511429
[346]	valid_0's l1: 511443
[347]	valid_0's l1: 511366
[348]	valid_0's l1: 511306
[349]	valid_0's l1: 511244
[

[623]	valid_0's l1: 502016
[624]	valid_0's l1: 501982
[625]	valid_0's l1: 501985
[626]	valid_0's l1: 501975
[627]	valid_0's l1: 502002
[628]	valid_0's l1: 501962
[629]	valid_0's l1: 501937
[630]	valid_0's l1: 501924
[631]	valid_0's l1: 501931
[632]	valid_0's l1: 501862
[633]	valid_0's l1: 501834
[634]	valid_0's l1: 501766
[635]	valid_0's l1: 501745
[636]	valid_0's l1: 501754
[637]	valid_0's l1: 501773
[638]	valid_0's l1: 501770
[639]	valid_0's l1: 501770
[640]	valid_0's l1: 501752
[641]	valid_0's l1: 501748
[642]	valid_0's l1: 501765
[643]	valid_0's l1: 501754
[644]	valid_0's l1: 501685
[645]	valid_0's l1: 501644
[646]	valid_0's l1: 501590
[647]	valid_0's l1: 501591
[648]	valid_0's l1: 501581
[649]	valid_0's l1: 501612
[650]	valid_0's l1: 501543
[651]	valid_0's l1: 501540
[652]	valid_0's l1: 501484
[653]	valid_0's l1: 501469
[654]	valid_0's l1: 501456
[655]	valid_0's l1: 501450
[656]	valid_0's l1: 501455
[657]	valid_0's l1: 501469
[658]	valid_0's l1: 501405
[659]	valid_0's l1: 501433
[

[933]	valid_0's l1: 497524
[934]	valid_0's l1: 497537
[935]	valid_0's l1: 497519
[936]	valid_0's l1: 497482
[937]	valid_0's l1: 497475
[938]	valid_0's l1: 497485
[939]	valid_0's l1: 497456
[940]	valid_0's l1: 497468
[941]	valid_0's l1: 497450
[942]	valid_0's l1: 497469
[943]	valid_0's l1: 497474
[944]	valid_0's l1: 497489
[945]	valid_0's l1: 497451
[946]	valid_0's l1: 497457
[947]	valid_0's l1: 497434
[948]	valid_0's l1: 497449
[949]	valid_0's l1: 497436
[950]	valid_0's l1: 497355
[951]	valid_0's l1: 497334
[952]	valid_0's l1: 497332
[953]	valid_0's l1: 497336
[954]	valid_0's l1: 497291
[955]	valid_0's l1: 497268
[956]	valid_0's l1: 497305
[957]	valid_0's l1: 497286
[958]	valid_0's l1: 497280
[959]	valid_0's l1: 497245
[960]	valid_0's l1: 497257
[961]	valid_0's l1: 497253
[962]	valid_0's l1: 497228
[963]	valid_0's l1: 497214
[964]	valid_0's l1: 497177
[965]	valid_0's l1: 497152
[966]	valid_0's l1: 497171
[967]	valid_0's l1: 497128
[968]	valid_0's l1: 497134
[969]	valid_0's l1: 497099
[

[1230]	valid_0's l1: 494993
[1231]	valid_0's l1: 494972
[1232]	valid_0's l1: 494965
[1233]	valid_0's l1: 494977
[1234]	valid_0's l1: 494987
[1235]	valid_0's l1: 495000
[1236]	valid_0's l1: 494990
[1237]	valid_0's l1: 495010
[1238]	valid_0's l1: 495000
[1239]	valid_0's l1: 495021
[1240]	valid_0's l1: 495021
[1241]	valid_0's l1: 494993
[1242]	valid_0's l1: 494989
[1243]	valid_0's l1: 494952
[1244]	valid_0's l1: 494950
[1245]	valid_0's l1: 494952
[1246]	valid_0's l1: 494930
[1247]	valid_0's l1: 494900
[1248]	valid_0's l1: 494853
[1249]	valid_0's l1: 494854
[1250]	valid_0's l1: 494837
[1251]	valid_0's l1: 494848
[1252]	valid_0's l1: 494857
[1253]	valid_0's l1: 494868
[1254]	valid_0's l1: 494890
[1255]	valid_0's l1: 494880
[1256]	valid_0's l1: 494826
[1257]	valid_0's l1: 494789
[1258]	valid_0's l1: 494821
[1259]	valid_0's l1: 494785
[1260]	valid_0's l1: 494806
[1261]	valid_0's l1: 494812
[1262]	valid_0's l1: 494834
[1263]	valid_0's l1: 494836
[1264]	valid_0's l1: 494844
[1265]	valid_0's l1:

[1537]	valid_0's l1: 493350
[1538]	valid_0's l1: 493348
[1539]	valid_0's l1: 493358
[1540]	valid_0's l1: 493358
[1541]	valid_0's l1: 493375
[1542]	valid_0's l1: 493373
[1543]	valid_0's l1: 493371
[1544]	valid_0's l1: 493375
[1545]	valid_0's l1: 493378
[1546]	valid_0's l1: 493341
[1547]	valid_0's l1: 493364
[1548]	valid_0's l1: 493344
[1549]	valid_0's l1: 493331
[1550]	valid_0's l1: 493295
[1551]	valid_0's l1: 493269
[1552]	valid_0's l1: 493284
[1553]	valid_0's l1: 493272
[1554]	valid_0's l1: 493272
[1555]	valid_0's l1: 493305
[1556]	valid_0's l1: 493310
[1557]	valid_0's l1: 493321
[1558]	valid_0's l1: 493312
[1559]	valid_0's l1: 493321
[1560]	valid_0's l1: 493323
[1561]	valid_0's l1: 493287
[1562]	valid_0's l1: 493266
[1563]	valid_0's l1: 493243
[1564]	valid_0's l1: 493238
[1565]	valid_0's l1: 493239
[1566]	valid_0's l1: 493222
[1567]	valid_0's l1: 493172
[1568]	valid_0's l1: 493163
[1569]	valid_0's l1: 493123
[1570]	valid_0's l1: 493062
[1571]	valid_0's l1: 493061
[1572]	valid_0's l1:

[1832]	valid_0's l1: 491687
[1833]	valid_0's l1: 491692
[1834]	valid_0's l1: 491694
[1835]	valid_0's l1: 491679
[1836]	valid_0's l1: 491698
[1837]	valid_0's l1: 491671
[1838]	valid_0's l1: 491648
[1839]	valid_0's l1: 491662
[1840]	valid_0's l1: 491663
[1841]	valid_0's l1: 491659
[1842]	valid_0's l1: 491655
[1843]	valid_0's l1: 491636
[1844]	valid_0's l1: 491618
[1845]	valid_0's l1: 491602
[1846]	valid_0's l1: 491612
[1847]	valid_0's l1: 491623
[1848]	valid_0's l1: 491601
[1849]	valid_0's l1: 491599
[1850]	valid_0's l1: 491595
[1851]	valid_0's l1: 491606
[1852]	valid_0's l1: 491590
[1853]	valid_0's l1: 491588
[1854]	valid_0's l1: 491579
[1855]	valid_0's l1: 491599
[1856]	valid_0's l1: 491572
[1857]	valid_0's l1: 491566
[1858]	valid_0's l1: 491548
[1859]	valid_0's l1: 491553
[1860]	valid_0's l1: 491549
[1861]	valid_0's l1: 491544
[1862]	valid_0's l1: 491554
[1863]	valid_0's l1: 491565
[1864]	valid_0's l1: 491558
[1865]	valid_0's l1: 491568
[1866]	valid_0's l1: 491575
[1867]	valid_0's l1:

In [22]:
Y_pred = reg.predict(X_val)
mean_absolute_error(Y_val,Y_pred)

487268.28239609225

In [331]:
# preparamos el csv de respuesta para kaggle

In [46]:
ids = test_selected.index.values
X_test = test_selected.values

In [47]:
test_predict = reg.predict(X_test)
escribir_respuesta(ids, test_predict)

In [ ]:
# best params so far
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mae',
    'max_depth': 14, 
    'learning_rate': 0.05,
    'verbose': 0, 
    'early_stopping_round': 200}
n_estimators=20000

### Modelo: KNN

### Modelo: Neural Networks

In [3]:
# ...